# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846874493114                   -0.70    4.0
  2   -7.852328586182       -2.26       -1.53    1.0
  3   -7.852646094623       -3.50       -2.52    3.2
  4   -7.852646676480       -6.24       -3.35    2.2
  5   -7.852646685781       -8.03       -4.76    1.0
  6   -7.852646686726       -9.02       -5.18    4.2
  7   -7.852646686729      -11.48       -5.85    2.0
  8   -7.852646686730      -12.42       -7.35    1.8
  9   -7.852646686730   +    -Inf       -7.72    2.8
 10   -7.852646686730   +  -14.45       -8.86    2.0
 11   -7.852646686730      -14.45      -10.04    3.2
 12   -7.852646686730   +  -14.75       -9.11    1.0
 13   -7.852646686730      -14.45       -9.11    1.0
 14   -7.852646686730   +  -14.45       -8.62    2.0
 15   -7.852646686730      -14.57       -9.23    2.0
 16   -7.852646686730   +  -15.05       -9.51    1.0
 17   -7.852646686730      -15.05      -10.22 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846782085513                   -0.70           4.2
  2   -7.852525402160       -2.24       -1.62   0.80    4.8
  3   -7.852607691406       -4.08       -2.79   0.80    5.0
  4   -7.852646566040       -4.41       -3.54   0.80    7.2
  5   -7.852646683253       -6.93       -4.44   0.80    9.0
  6   -7.852646686645       -8.47       -5.06   0.80   10.5
  7   -7.852646686725      -10.10       -6.25   0.80   11.5
  8   -7.852646686730      -11.29       -7.18   0.80   13.5
  9   -7.852646686730      -13.55       -7.58   0.80   15.5
 10   -7.852646686730      -14.75       -9.13   0.80   16.0
 11   -7.852646686730      -14.75       -9.20   0.80   19.0
 12   -7.852646686730   +  -14.57      -10.42   0.80   19.0
 13   -7.852646686730   +    -Inf      -11.24   0.80   20.5
 14   -7.852646686730   +  -15.05      -12.10   0.80   23.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.387852e+01     4.218717e+00
 * time: 0.5102279186248779
     1     1.178744e+00     2.161491e+00
 * time: 0.788917064666748
     2    -1.811828e+00     2.511598e+00
 * time: 0.8178238868713379
     3    -3.958697e+00     2.204426e+00
 * time: 0.864048957824707
     4    -5.502421e+00     1.701293e+00
 * time: 0.9067339897155762
     5    -7.025697e+00     9.972730e-01
 * time: 0.949289083480835
     6    -7.174468e+00     1.238617e+00
 * time: 0.9775879383087158
     7    -7.560807e+00     9.737606e-01
 * time: 1.007253885269165
     8    -7.585624e+00     1.088954e+00
 * time: 1.0363359451293945
     9    -7.742058e+00     6.784993e-01
 * time: 1.0687808990478516
    10    -7.791830e+00     4.637104e-01
 * time: 1.097316026687622
    11    -7.836117e+00     5.859185e-02
 * time: 1.1267640590667725
    12    -7.844831e+00     7.372138e-02
 * time: 1.1555581092834473
    13    -7.850437e+00     3.903608e-02
 * time: 1.188498020172119

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846755176189                   -0.70    4.0
  2   -7.852307783548       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686679                   -2.54
  2   -7.852646686730      -10.30       -5.77
  3   -7.852646686730      -15.05      -12.17
|ρ_newton - ρ_scf|  = 3.5147992433808814e-13
|ρ_newton - ρ_scfv| = 3.9477307655799686e-13
|ρ_newton - ρ_dm|   = 5.82065555729629e-10
